In [3]:
import numpy as np
import cv2
import time
from scipy.fft import fft2, ifft2

In [6]:
# Tent map implementation
def tent_map(x, p, iterations, size):
    """Generate a pseudo-random image using the tent map."""
    result = np.zeros(size)
    for i in range(size[0]):
        for j in range(size[1]):
            x_n = x
            for _ in range(iterations):
                if 0 <= x_n <= p:
                    x_n = x_n / p
                elif p < x_n <= 1:
                    x_n = (1 - x_n) / (1 - p)
            result[i, j] = x_n
    return result

# Bernoulli map implementation
def bernoulli_map(x, b, iterations, size):
    """Generate a pseudo-random image using the Bernoulli map."""
    result = np.zeros(size)
    for i in range(size[0]):
        for j in range(size[1]):
            x_n = x
            for _ in range(iterations):
                x_n = (b * x_n) % 1
            result[i, j] = x_n
    return result

# Phase-magnitude transformation unit
def phase_magnitude_transformation(image, key1, p, iterations):
    """Apply chaotic phase-magnitude transformation using tent map."""
    # Perform 2D DFT
    f_transform = fft2(image)
    magnitude = np.abs(f_transform)
    phase = np.angle(f_transform)

    # Generate pseudo-random image using tent map
    tent_image = tent_map(key1, p, iterations, image.shape)

    # Perform 2D DFT on tent image
    tent_transform = fft2(tent_image)
    tent_magnitude = np.abs(tent_transform)
    tent_phase = np.angle(tent_transform)

    # Linear combination for new phase and magnitude (Eq. 8 and 9)
    new_phase = (phase + 5 * tent_phase) / 6
    new_magnitude = (magnitude + 5 * tent_magnitude) / 6

    # Reconstruct complex Fourier coefficients
    new_transform = new_magnitude * np.exp(1j * new_phase)

    # Inverse 2D DFT
    encrypted_image = np.abs(ifft2(new_transform))
    return encrypted_image

# Pixel substitution unit
def pixel_substitution(encrypted_image, key2, b, iterations):
    """Apply chaotic pixel substitution using Bernoulli map."""
    # Generate pseudo-random image using Bernoulli map
    bernoulli_image = bernoulli_map(key2, b, iterations, encrypted_image.shape)

    # Normalize and quantize Bernoulli output to [0, 255]
    bernoulli_image = (bernoulli_image * 256).astype(np.float32)
    bernoulli_image = np.clip(bernoulli_image, 0, 255).astype(np.uint8)

    # Square and modular addition (Eq. 10)
    bernoulli_squared = (bernoulli_image ** 2).astype(np.uint16)  # Use uint16 to avoid overflow
    final_encrypted = ((bernoulli_squared + encrypted_image.astype(np.uint16)) % 256).astype(np.uint8)
    return final_encrypted

# Decryption functions
def inverse_pixel_substitution(encrypted_image, key2, b, iterations):
    """Reverse the pixel substitution step."""
    bernoulli_image = bernoulli_map(key2, b, iterations, encrypted_image.shape)
    bernoulli_image = (bernoulli_image * 256).astype(np.float32)
    bernoulli_image = np.clip(bernoulli_image, 0, 255).astype(np.uint8)
    bernoulli_squared = (bernoulli_image ** 2).astype(np.uint16)
    decrypted_image = ((encrypted_image.astype(np.uint16) - bernoulli_squared) % 256).astype(np.uint8)
    return decrypted_image

def inverse_phase_magnitude_transformation(image, key1, p, iterations):
    """Reverse the phase-magnitude transformation step."""
    f_transform = fft2(image)
    magnitude = np.abs(f_transform)
    phase = np.angle(f_transform)

    tent_image = tent_map(key1, p, iterations, image.shape)
    tent_transform = fft2(tent_image)
    tent_magnitude = np.abs(tent_transform)
    tent_phase = np.angle(tent_transform)

    # Reverse the linear combination
    original_phase = (6 * phase - 5 * tent_phase)
    original_magnitude = (6 * magnitude - 5 * tent_magnitude)

    original_transform = original_magnitude * np.exp(1j * original_phase)
    decrypted_image = np.abs(ifft2(original_transform))
    return decrypted_image

# Main encryption function
def encrypt_image(image_path, key1=0.4, p=0.3, key2=0.5, b=1.5, iterations=10):
    """Encrypt the input image and measure encryption time."""
    # Load image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError("Image not found or invalid format")

    start_time = time.time()

    # Phase-magnitude transformation
    encrypted_image1 = phase_magnitude_transformation(image, key1, p, iterations)

    # Pixel substitution
    final_encrypted = pixel_substitution(encrypted_image1, key2, b, iterations)

    encryption_time = time.time() - start_time

    return final_encrypted, encryption_time

# Main decryption function
def decrypt_image(encrypted_image, key1=0.4, p=0.3, key2=0.5, b=1.5, iterations=10):
    """Decrypt the input image and measure decryption time."""
    start_time = time.time()

    # Reverse pixel substitution
    decrypted_image1 = inverse_pixel_substitution(encrypted_image, key2, b, iterations)

    # Reverse phase-magnitude transformation
    final_decrypted = inverse_phase_magnitude_transformation(decrypted_image1, key1, p, iterations)

    decryption_time = time.time() - start_time

    return final_decrypted, decryption_time

In [7]:
# Example usage
if __name__ == "__main__":
    image_paths = ['baboon.png', 'apple.png', 'jupiter.png', 'cameraman.png', 'medical.png']
    key1, p = 0.4, 0.3  # Tent map parameters
    key2, b = 0.5, 1.5  # Bernoulli map parameters
    iterations = 10

    for image_path in image_paths:
        try:
            # Encrypt
            encrypted_image, enc_time = encrypt_image(image_path, key1, p, key2, b, iterations)
            print(f"Encryption time for {image_path}: {enc_time:.4f} seconds")

            # Save encrypted image
            cv2.imwrite(f"encrypted_{image_path}", encrypted_image)

            # Decrypt
            decrypted_image, dec_time = decrypt_image(encrypted_image, key1, p, key2, b, iterations)
            print(f"Decryption time for {image_path}: {dec_time:.4f} seconds")

            # Save decrypted image
            cv2.imwrite(f"decrypted_{image_path}", decrypted_image)

        except Exception as e:
            print(f"Error processing {image_path}: {str(e)}")

Encryption time for baboon.png: 1.0717 seconds
Decryption time for baboon.png: 1.6108 seconds


[ WARN:0@119.363] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


Encryption time for apple.png: 0.2187 seconds
Decryption time for apple.png: 0.2171 seconds
Encryption time for jupiter.png: 0.4165 seconds
Decryption time for jupiter.png: 0.4254 seconds
Encryption time for cameraman.png: 0.2774 seconds
Decryption time for cameraman.png: 0.2562 seconds
Encryption time for medical.png: 0.2490 seconds
Decryption time for medical.png: 0.2347 seconds
